# Decoder・AuxLoss モジュールの解説と実装

## Decoder および AuxLoss モジュールの構造
Decoder モジュールや AuxLoss モジュールは，Pyramid Pooling モジュールまたは Feature モジュールの出力を Decode しピクセルごとに物体ラベルをクラス分類で推定し，最後に画像サイズを元の 475×475 に UpSample する．  
それぞれのモジュールの構成を次に示す．両者はともに同じネットワーク構造をしている．

<img src="../image/p163.png">

conv2DBatchNormRelu クラスを通過し，ドロップアウト層を通したうえで，もう1度畳み込み層にかけることで，21×60×60 の出力を得る．
最後に UpSample 層を通って 475×475 に拡大する．  
最終的なモジュールの出力は 21×475×475 になる．
出力テンソルの各値は，各ピクセルの21クラスに対する確信度を表しており，その値の最も大きなクラスがそのピクセルの物体ラベルとなる．

## Decoder および AuxLoss モジュールの実装

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DecodePSPFeature(nn.Module):
    def __init__(self, height, width, n_classes):
        super(DecodePSPFeature, self).__init__()
        
        # forward で使用する画像サイズ
        self.height = height
        self.width = width
        
        self.cbr = conv2DBatchNormRelu(in_channels=4096, out_channels=512, kernel_size=3, stride=1, dilation=1, bias=False)
        self.dropout = nn.Dropout2d(p=0.1)
        self.classification = nn.Conv2d(in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x):
        x = self.cbr(x)
        x = self.dropout(x)
        x = self.classification(x)
        output = F.interpolate(x, size=(self.height, self.width), mode="bilinear", align_corners=True)
        
        return output
    

class AuxiliaryPSPlayers(nn.Module):
    def __init__(self, in_channels, height, width, n_classes):
        super(AuxiliaryPSPlayers, self).__init__()
        
        # forward で使用する画像サイズ
        self.height = height
        self.width = width
        
        self.cbr = conv2DBatchNormRelu(in_channels=in_channels, out_channels=256, kernel_size=3, stride=1, dilation=1, bias=False)
        self.dropout = nn.Dropout2d(p=0.1)
        self.classification = nn.Conv2d(in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
        
    def forward(self, x):
        x = self.cbr(x)
        x = self.dropout(x)
        x = self.classification(x)
        output = F.interpolate(x, size=(self.height, self.width), mode="bilinear", align_corners=True)
        
        return output

詳細は第5章で触れるが，self.classification のように全結合層ではなくクラス数と同じチャネル数を出力するカーネルサイズ1の畳み込み層を pointwise convolution と呼ぶ．
最後に PSPNet のインスタンスが正しく作成できることを確認しておく．

In [1]:
import utils.pspnet as pspnet

net = pspnet.PSPNet(n_classes=21)
print(net)

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec